In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mm_functions as mmf
import params as pr
from scipy import interpolate

In [ ]:
# Data location
# prefix_dir = '/scratch2/raphaelr/processed_data/'
# sigpref = '/scratch2/raphaelr/processed_data/significance_maps/'
prefix_dir = '/Users/raphael/Documents/work/data/'
sigpref = '/Users/raphael/Documents/work/data/significance_maps/'

# Vectors for plotting
lon_qvec = np.linspace(-pr.half_width,pr.half_width,pr.half_width*pr.eff_factor+1)
lat_qvec = np.linspace(-pr.half_width,pr.half_width,pr.half_width*pr.eff_factor+1)
# Vectors for plotting HR
lon_qvec_hr = np.linspace(-pr.half_width,pr.half_width,pr.half_width*pr.res_factor+1)
lat_qvec_hr = np.linspace(-pr.half_width,pr.half_width,pr.half_width*pr.res_factor+1)

In [ ]:
cluster_vec = np.arange(1,9+1)
var_lists = [['precip','wind_mag'],['precip_mask','swh'],['pm10','t2m']]
name_lists = [['Rain','Wind'],['Rain','Wave'],['pm10','Heat']]
mm_list = [[0,1],[0,1,2],[0,1,3],[0,1,4]]

In [ ]:
cluster_vec = np.arange(1,9+1)
var_lists = [['precip'],['wind_mag'],['swh'],['pm10']]
name_lists = [['Rain'],['Wind'],['Wave'],['pm10']]
mm_list = [[0],[0,1],[0,2],[0,3]]

In [ ]:
for vv, var_list in enumerate(var_lists):

    name_list = name_lists[vv]
    
    # Variables will always be treated in that order!
    feat_list = ['WCB','cold_front','DI']
    var_feat_list = var_list + feat_list

    print(var_feat_list)
    n_clus = len(cluster_vec)
    n_mm = len(mm_list)
    summary_var = np.zeros((n_clus,n_mm))

    for ii, cluster_number in enumerate(cluster_vec):   
        # Make MM composite
        mm_comp_list, dens_list  = mmf.make_composite(var_feat_list,mm_list,prefix_dir,cluster_number,'plot')

        # Define significance map only for the first input variable
        analysis_name = "-".join(list(map(var_feat_list.__getitem__, mm_list[0])))
        sigdir= sigpref+'C'+str(cluster_number)+'-'+analysis_name+'.npy'
        sigmap = np.load(sigdir)

        trunc = []
        # Now loop through, producing the summary variables
        for jj, mm_comp in enumerate(mm_comp_list):     

            # Setting to zero wherever not significant
            mm_comp_list[jj][sigmap]=0

            trunc.append(mm_comp_list[jj][40:121,40:121])
#             trunc.append(mm_comp_list[jj])
    
            # Computing number of point to normalize by (should not change even if recomputed)
            npts = (trunc[jj].shape[0]*trunc[jj].shape[1])

            # Computing summary variable
            summary_var[ii,jj] = np.sum(trunc[jj])/npts

    sort_vec = np.argsort(summary_var[:,0])
    sort_vec = sort_vec[::-1]
    clus_label = [str(x) for x in (sort_vec+1)]


    
    if vv == 0: fig, axs = plt.subplots(len(var_lists),1)

    ind = cluster_vec
    ax = axs[vv]
    ax.bar(ind, 100*summary_var[sort_vec,0], width=0.8,facecolor='white',edgecolor='k')
    ax.bar(ind-0.8/3, 100*summary_var[sort_vec,1],width=0.8/3,color='black')
    ax.bar(ind, 100*summary_var[sort_vec,2],width=0.8/3,color=(0.4, 0.4, 0.4))
    ax.bar(ind+0.8/3, 100*summary_var[sort_vec,3],width=0.8/3,color=(0.7, 0.7, 0.7))
    ax.set_xticks(ind, labels=clus_label)
    ax.set_ylabel('Freq. of occurrence [%]')
    atitle = "-".join(list(map(name_list.__getitem__, mm_list[0])))
    ax.set_title(atitle+' and Dyn. Features')
    if vv == len(var_lists)-1: ax.set_xlabel('Cluster ID')
    if vv == 1: ax.legend(['Hazard','Hazard-WCB','Hazard-CF','Hazard-DI'])
    
plt.subplots_adjust(wspace = 0.3, hspace=0.35, left=0.15)

fig.set_size_inches(5,8/3*len(var_lists))
# fig.savefig('Features_compound.pdf')

In [ ]:
# Variables will always be treated in that order!
mm_list = [[0],[1],[0,1],[0,1,2]]

n_clus = len(cluster_vec)
n_mm = len(mm_list)
summary_var = np.zeros((n_clus,n_mm+2))
for ii, cluster_number in enumerate(cluster_vec):   
    # Make MM composite
    mm_comp_list, dens_list  = mmf.make_composite(var_list,mm_list,prefix_dir,cluster_number,'plot')
    
    # Load significance maps for all variables
    sigmap_part = []    
    for gg, mm_comp in enumerate(mm_comp_list):     
        # Name of the analysis and loading of the significance map
        analysis_name = "-".join(list(map(var_list.__getitem__, mm_list[gg])))
        sigdir= sigpref+'C'+str(cluster_number)+'-'+analysis_name+'.npy'
        sigmap_part.append(np.load(sigdir))
     
    # Redefine, including the expanded definition 
    # This is absolutely beautiful!
    sigmap = []
    sigmap.append(~np.logical_or(~sigmap_part[0],~sigmap_part[2])) # [0]
    sigmap.append(~np.logical_or(~sigmap_part[1],~sigmap_part[2])) # [1]
    sigmap.append(sigmap_part[2]) # [2]
    
    trunc = []
    # Now loop through, producing the summary variables
    for jj, mm_comp in enumerate(mm_comp_list):     
        
        # Setting to zero wherever not significant
        mm_comp_list[jj][sigmap[jj]]=0
        
        trunc.append(mm_comp_list[jj][40:121,40:121])
        
        # Computing number of point to normalize by (should not change even if recomputed)
        npts = (trunc[jj].shape[0]*trunc[jj].shape[1])
        
        # Computing summary variable
        summary_var[ii,jj] = np.sum(trunc[jj])/npts
        
    # Just an additional test - may be more than 100% because compound may be significant even when individual hazards are not...
    # May be fixed by adding the individual variable sig. region AND the compound variable sig. region.
    tmp = np.minimum(trunc[0],trunc[1])
    summary_var[ii,-2] = np.sum(tmp)/npts
    
    # Annnnd just one more
    sum_list = []
    for hh, var_name in enumerate(var_list):
        fn = prefix_dir+var_name+'/'+var_name+'_hr_bool_c'+str(cluster_number)+'.npz'
        tmp = np.load(fn)
        tmp_comp = tmp['comp']
        sigdir= sigpref+'C'+str(cluster_number)+'-'+var_name+'.npy'
        sigarray2D = sigmap[hh]
        sigarray3D = np.repeat(np.expand_dims(sigarray2D, 2),tmp_comp.shape[2], axis=2)
        tmp_comp[sigarray3D] = 0
        
        tmp_trunc = tmp_comp[40:121,40:121,:]
        
#         densarray3D = np.repeat(np.expand_dims(dens_list[hh]/tmp_comp.shape[2], 2),tmp_comp.shape[2], axis=2)
        densarray3D = np.expand_dims(dens_list[hh]/tmp_comp.shape[2], 2)
        
        dens_trunc = densarray3D[40:121,40:121,:]
        
        # Normalizing variable
#         norm_var[hh] = np.sum(dens_list[hh][~sigmap[hh]])/np.sum(~sigmap[hh])/tmp_comp.shape[2]
        
        sum_list.append(np.nansum(tmp_trunc/dens_trunc,axis=(0,1)))
    # Select the shortest length to compare arrays
    len1 = len(sum_list[0])
    len2 = len(sum_list[1])
    eff_len = np.minimum(len1,len2)    
    min_vect = np.minimum(sum_list[0][-eff_len:],sum_list[1][-eff_len:]) 
    summary_var[ii,-1] = np.mean(min_vect)/npts

In [ ]:
sort_vec = np.argsort(summary_var[:,2])
sort_vec = sort_vec[::-1]
clus_label = [str(x) for x in (sort_vec+1)]

# 0 is precipitation occurrence, 1 is wind occurrence 2 is actual compound occurrence
# 3 is occurrence if temporal (but not spatial) correpondence was perfect
# 4 is occurrence if spatial (but not temporal) correspondence was perfect
# This variable is if both temporal and spatial compounding were perfect
perfect_comp = np.minimum(summary_var[sort_vec,0],summary_var[sort_vec,1])

In [ ]:
# Weighted frequency
w_vect = np.array([0.16, 0.12, 0.07, 0.2, 0.08, 0.09, 0.11, 0.09, 0.09])
frac_vect = summary_var[sort_vec,2]*w_vect[sort_vec]

In [ ]:

fig, axs = plt.subplots(3,1)
ind = cluster_vec
ax = axs[0]
ax.bar(ind, 100*summary_var[sort_vec,2], width=0.8,color='purple')
ax.set_xticks(ind, labels=clus_label)
ax.set_ylabel('Freq. of occurrence [%]')
ax.set_title('Compound event')

ax = axs[1]
ax.bar(ind, 100*summary_var[sort_vec,0], width=0.8,color='blue')
ax.set_xticks(ind, labels=clus_label)
ax.set_ylabel('Freq. of occurrence [%]')
ax.set_title(name_list[0]+' event')

ax = axs[2]
ax.bar(ind, 100*summary_var[sort_vec,1], width=0.8,color='red')
ax.set_xticks(ind, labels=clus_label)
ax.set_xlabel('Cluster ID')
ax.set_ylabel('Freq. of occurrence [%]')
ax.set_title(name_list[1]+' event')

plt.subplots_adjust(wspace = 0.3, hspace=0.35, left=0.15)
fig.set_size_inches(5,8)
fig.savefig('Compound_diagnostics_'+name_list[0]+'_'+name_list[1]+'.pdf')

In [ ]:
fig, axs = plt.subplots(3,1)
ind = cluster_vec

ax=axs[0]
ax.bar(ind, 100*perfect_comp, width=0.8,color='grey')
ax.bar(ind, 100*summary_var[sort_vec,2], width=0.8, color='purple')
ax.set_xticks(ind, labels=clus_label)
ax.set_ylabel('Freq. of occurrence [%]')
ax.set_title('Ideal compound scenario')

ax=axs[1]
ax.bar(ind, 100*summary_var[sort_vec,2]/summary_var[sort_vec,3], width=0.8,color='grey')
ax.set_xlim(0,10)
ax.set_xticks(ind, labels=clus_label)
ax.set_ylabel('Simultaneity [%]')
ax.set_title('Simultaneity of '+name_list[0]+' and '+name_list[1]+' events')

ax=axs[2]
ax.bar(ind, 100*summary_var[sort_vec,2]/summary_var[sort_vec,4], width=0.8,color='grey')
ax.set_xlim(0,10)
ax.set_xticks(ind, labels=clus_label)
ax.set_xlabel('Cluster ID')
ax.set_ylabel('Overlap [%]')
ax.set_title('Spatial overlap of '+name_list[0]+' and '+name_list[1]+' events')

plt.subplots_adjust(wspace = 0.3, hspace=0.35, left=0.15)
fig.set_size_inches(5,8)
fig.savefig('Compound_ideal_'+name_list[0]+'_'+name_list[1]+'.pdf')